In [6]:
#   %%writefile Dec_BRIDGE.py
# Decentralized learning with BRIDGE

# Specify M nodes, N local samples, dataset, failure type, screening type, stepsize, connection rate, T iterations,
# b Byzantine agents

import numpy as np
from pre_proc import data_prep
from linear_classifier import linear_classifier
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
    def __init__(self, agents, dataset, localsize_N, iteration, stepsize=1e-4, 
                 screen=False, b=0, Byzantine='random', connection_rate=50):
        self.dataset = dataset
        self.M = agents
        self.T = iteration
        self.screen = screen
        self.b = b
        self.stepsize = stepsize
        self.N = localsize_N
        self.Byzantine = Byzantine
        self.con_rate = connection_rate

def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
    
    re = 1                      # regenerate if graph assumption not satisfied
    while re:
        graph = []
        for _ in range(nodes):
            graph.append([])
        for row in range(nodes):
            graph[row].append(1)
            for col in range(row + 1, nodes):
                d = random.randint(1, 100)
                if d < con_rate:
                    graph[row].append(1)     #form symmetric matrix row by row
                    graph[col].append(1)
                else:
                    graph[row].append(0)
                    graph[col].append(0)
        d_max = 0
        for row in graph:
            if sum(row) > d_max:
                d_max = sum(row)
        w = [row[:] for row in graph]
        for ind, row in enumerate(w):
            d = sum(row)
            w[ind] = [col/d_max for col in row]
            w[ind][ind] = 1 - (d - 1) / d_max
        if all([sum(row) >= 2 * b + 1 for row in graph]):
            re = 0
    return w, graph   

def get_neighbor(G):
    neighbor_list = []
    for node in G:
        neighbor = []
        for ind, n in enumerate(node):
            if n == 1:
                neighbor.append(ind)
        neighbor_list.append(neighbor)
    return neighbor_list

def one_hot(label):
    l_oh = []
    for i in label:
        new_l = [0] * 10
        new_l[i] = 1
        l_oh.append(new_l)
    return l_oh


def initialization():
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    return sess


def screen(grad, b, method):
    screened = Byzantine_algs[method](grad, b)
    return screened
    
def acc_test(model, t_data, t_label):
    acc = model.accuracy.eval(feed_dict={
            model.x:t_data, model.y_: t_label})
    return acc
    
def communication(W, neighbor, sess, b=0, screen=False):
    wb = [node.weights() for node in W]
    ave_w = []
    ave_b = []
    for neighbor_list in neighbor:
        neighborhood_w = [wb[n][0] for n in neighbor_list]
        neighborhood_b = [wb[n][1] for n in neighbor_list] 
        if screen:
            neighborhood_w = np.sort(neighborhood_w, axis = 0) 
            neighborhood_w = neighborhood_w[b : -b]
            neighborhood_b = np.sort(neighborhood_b, axis = 0)
            neighborhood_b = neighborhood_b[b : -b]
        neighborhood_w = np.mean(neighborhood_w, axis = 0)
        neighborhood_b = np.mean(neighborhood_b, axis = 0)
    #        print(neighborhood.shape)
        ave_w.append(neighborhood_w)
        ave_b.append(neighborhood_b)

    for node, w, b in zip(W, ave_w, ave_b):
        node.assign([w, b], sess) 

def node_update(W, data, sess, stepsize=1e-3):        
    for model, sample, label in zip(W, data.dist_data, data.dist_label):
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
        




In [ ]:
if __name__ == "__main__":
    para = experiment_parameters(agents=20, dataset='MNIST', localsize_N=2000, iteration=1000,
                                   screen=False, b=0, Byzantine='random', stepsize = 1e-1)
    #Generate the graph
    W_0, graph = gen_graph(para.M, para.con_rate, para.b)    
    local_set, test_data, test_label = data_prep(para.dataset, para.M, para.N, one_hot=True)
    neighbors = get_neighbor(graph)
    #Initialization
    tf.reset_default_graph()
    w_nodes = [linear_classifier(stepsize = para.stepsize) for node in range(para.M)]    
    sess = initialization()
    rec = []
 
    for iteration in range(para.T):
        #Communication 
        communication(w_nodes, neighbors, sess, para.b, para.screen)
        
#         if iteration%10 == 0:            
        #test over all test data
        accuracy = [acc_test(node, test_data, test_label) for node in w_nodes]
#             print(accuracy)
        rec.append(np.mean(accuracy))
        #node update using GD   
        node_update(w_nodes, local_set, sess, stepsize=para.stepsize/(iteration+1))

    
    sess.close()
    with open('./result/result_DGD_faultless.pickle', 'wb') as handle:
        pickle.dump(rec, handle)

In [11]:
# local_set, test_data, test_label = data_prep(para.dataset, para.M, para.N, one_hot=True)
# data_2k = test_data[:2000]
# label_2k = test_label[:2000]
# tf.reset_default_graph()
# w_2k = linear_classifier(stepsize=1e-1)
# sess = initialization()
# for ieteration in range(1000):
#     sess.run(w_2k.train_step, feed_dict={w_2k.x: data_2k, w_2k.y_: label_2k, w_2k.stepsize: 1e-1})
acc = w_2k.accuracy.eval(feed_dict={
            w_2k.x:test_data[2000:], w_2k.y_: test_label[2000:]})   

print(acc)


0.859375


In [1]:
 %%writefile dec_ByRDiE.py
# Decentralized learning with ByRDiE

# Specify M nodes, N local samples, dataset, failure type, screening type, stepsize, connection rate, T iterations,
# b Byzantine agents

import numpy as np
from pre_proc import data_prep
from linear_classifier import linear_classifier
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
    def __init__(self, agents, dataset, localsize_N, iteration, stepsize=1e-4, 
                 screen=False, b=0, Byzantine='random', connection_rate=50):
        self.dataset = dataset
        self.M = agents
        self.T = iteration
        self.screen = screen
        self.b = b
        self.stepsize = stepsize
        self.N = localsize_N
        self.Byzantine = Byzantine
        self.con_rate = connection_rate

def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
    
    re = 1                      # regenerate if graph assumption not satisfied
    while re:
        graph = []
        for _ in range(nodes):
            graph.append([])
        for row in range(nodes):
            graph[row].append(1)
            for col in range(row + 1, nodes):
                d = random.randint(1, 100)
                if d < con_rate:
                    graph[row].append(1)     #form symmetric matrix row by row
                    graph[col].append(1)
                else:
                    graph[row].append(0)
                    graph[col].append(0)
        d_max = 0
        for row in graph:
            if sum(row) > d_max:
                d_max = sum(row)
        w = [row[:] for row in graph]
        for ind, row in enumerate(w):
            d = sum(row)
            w[ind] = [col/d_max for col in row]
            w[ind][ind] = 1 - (d - 1) / d_max
        if all([sum(row) >= 2 * b + 1 for row in graph]):
            re = 0
    return w, graph   

def get_neighbor(G):
    neighbor_list = []
    for node in G:
        neighbor = []
        for ind, n in enumerate(node):
            if n == 1:
                neighbor.append(ind)
        neighbor_list.append(neighbor)
    return neighbor_list

def one_hot(label):
    l_oh = []
    for i in label:
        new_l = [0] * 10
        new_l[i] = 1
        l_oh.append(new_l)
    return l_oh


def initialization():
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    return sess


def screen(grad, b, method):
    screened = Byzantine_algs[method](grad, b)
    return screened
    
def acc_test(model, t_data, t_label):
    acc = model.accuracy.eval(feed_dict={
            model.x:t_data, model.y_: t_label,})
    return acc
    
def communication_w(W, neighbor, p, sess, b=0, screen=False):
    _w = [node.weights()[0].reshape([7840,1]) for node in W]
    _b = [node.weights()[1] for node in W]
    ave = []
    for neighbor_list in neighbor:
        neighborhood = [_w[n][p] for n in neighbor_list]

        if screen:
            neighborhood = np.sort(neighborhood, axis = 0)
            neighborhood = neighborhood[b : -b]
        neighborhood = np.mean(neighborhood, axis = 0)
    #        print(neighborhood.shape)
        ave.append(neighborhood)
    for scalar, node in zip(ave, _w):
        node[p] = scalar
    
    for node, ww, bb in zip(W, _w, _b):
        node.assign([ww.reshape([784, 10]), bb], sess) 
        
def communication_b(W, neighbor, p, sess, b=0, screen=False):
    _w = [node.weights()[0] for node in W]
    _b = [node.weights()[1] for node in W]
    ave = []
    for neighbor_list in neighbor:
        neighborhood = [_b[n][p] for n in neighbor_list]

        if screen:
            neighborhood = np.sort(neighborhood, axis = 0)
            neighborhood = neighborhood[b : -b]
        neighborhood = np.mean(neighborhood, axis = 0)
    #        print(neighborhood.shape)
        ave.append(neighborhood)
    for scalar, node in zip(ave, _b):
        node[p] = scalar
    
    for node, ww, bb in zip(W, _w, _b):
        node.assign([ww, bb], sess) 

def node_update_w(W, data, p, sess, stepsize=1e-4):        
    for model, sample, label in zip(W, data.dist_data, data.dist_label):
        g = sess.run(model.gradient_w, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
        new_g = np.zeros([7840, 1])
        new_g[p] = g.reshape([7840, 1])[p]
        sess.run(model.local_update_w, feed_dict={model.gradient_port_w: new_g.reshape([784,10]), model.stepsize: stepsize})
        
def node_update_b(W, data, p, sess, stepsize=1e-4):        
    for model, sample, label in zip(W, data.dist_data, data.dist_label):
        g = sess.run(model.gradient_b, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
        new_g = np.zeros([10])
        new_g[p] = g[p]
        sess.run(model.local_update_b, feed_dict={model.gradient_port_b: new_g, model.stepsize: stepsize})
        


if __name__ == "__main__":
    para = experiment_parameters(agents=20, dataset='MNIST', localsize_N=2000, iteration=100,
                                   screen=False, b=0, Byzantine='random', stepsize = 1e-1)
    #Generate the graph
    W_0, graph = gen_graph(para.M, para.con_rate, para.b)    
    local_set, test_data, test_label = data_prep(para.dataset, para.M, para.N, one_hot=True)
    neighbors = get_neighbor(graph)
    #Initialization
    tf.reset_default_graph()
    w_nodes = [linear_classifier(stepsize = para.stepsize) for node in range(para.M)]    
    sess = initialization()
    
    save = []
    for iteration in range(para.T):
        step = para.stepsize/(iteration+1)
        for p in range(7840):
        #Communication 
            communication_w(w_nodes, neighbors, p, sess, para.b, screen = para.screen)
        #node update using Adam    
            node_update_w(w_nodes, local_set, p, sess, stepsize=step)
            if p%200 == 199:            
            #test over all test data
                accuracy = [acc_test(node, test_data, test_label) for node in w_nodes]
                print(accuracy)
                save.append(accuracy)
        for p in range(10):
            communication_b(w_nodes, neighbors, p, sess, para.b, screen = para.screen)
            node_update_b(w_nodes, local_set, p, sess, stepsize=step)
        accuracy = [acc_test(node, test_data, test_label) for node in w_nodes]
        print(accuracy)
        print(iteration)
        save.append(accuracy)
        with open('./ByRDiE_faultless_0.pickle', 'wb') as handle:
            pickle.dump(save, handle) 
    sess.close()


Overwriting dec_ByRDiE.py


In [1]:
#   %%writefile Dec_BRIDGE.py
# Decentralized learning with BRIDGE

# Specify M nodes, N local samples, dataset, failure type, screening type, stepsize, connection rate, T iterations,
# b Byzantine agents

import numpy as np
from pre_proc import data_prep
from linear_classifier import linear_classifier
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
    def __init__(self, agents, dataset, localsize_N, iteration, stepsize=1e-4, 
                 screen=False, b=0, Byzantine='random', connection_rate=50):
        self.dataset = dataset
        self.M = agents
        self.T = iteration
        self.screen = screen
        self.b = b
        self.stepsize = stepsize
        self.N = localsize_N
        self.Byzantine = Byzantine
        self.con_rate = connection_rate

def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
    
    re = 1                      # regenerate if graph assumption not satisfied
    while re:
        graph = []
        for _ in range(nodes):
            graph.append([])
        for row in range(nodes):
            graph[row].append(1)
            for col in range(row + 1, nodes):
                d = random.randint(1, 100)
                if d < con_rate:
                    graph[row].append(1)     #form symmetric matrix row by row
                    graph[col].append(1)
                else:
                    graph[row].append(0)
                    graph[col].append(0)
        d_max = 0
        for row in graph:
            if sum(row) > d_max:
                d_max = sum(row)
        w = [row[:] for row in graph]
        for ind, row in enumerate(w):
            d = sum(row)
            w[ind] = [col/d_max for col in row]
            w[ind][ind] = 1 - (d - 1) / d_max
        if all([sum(row) >= 2 * b + 1 for row in graph]):
            re = 0
    return w, graph   

def get_neighbor(G):
    neighbor_list = []
    for node in G:
        neighbor = []
        for ind, n in enumerate(node):
            if n == 1:
                neighbor.append(ind)
        neighbor_list.append(neighbor)
    return neighbor_list

def one_hot(label):
    l_oh = []
    for i in label:
        new_l = [0] * 10
        new_l[i] = 1
        l_oh.append(new_l)
    return l_oh


def initialization():
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    return sess


def screen(grad, b, method):
    screened = Byzantine_algs[method](grad, b)
    return screened
    
def acc_test(model, t_data, t_label):
    acc = model.accuracy.eval(feed_dict={
            model.x:t_data, model.y_: t_label,})
    return acc
    
def communication(W, neighbor, sess, b=0, screen=False):
    wb = [node.weights() for node in W]
    # Byznatine nodes
    for node in range(b):
        wb[node][0] = np.random.rand(784,10) * 20
        wb[node][1] = np.random.rand(10) * 20
    ave_w = []
    ave_b = []
    for neighbor_list in neighbor:
        neighborhood_w = [wb[n][0] for n in neighbor_list]
        neighborhood_b = [wb[n][1] for n in neighbor_list] 
        if screen:
            #BRIDGE screening
#             neighborhood_w = np.sort(neighborhood_w, axis = 0) 
#             neighborhood_w = neighborhood_w[b : -b]
#             neighborhood_b = np.sort(neighborhood_b, axis = 0)
#             neighborhood_b = neighborhood_b[b : -b]
            #Median screening
            _M = int(len(neighborhood_w)/2)
            neighborhood_w = np.sort(neighborhood_w, axis = 0) 
            neighborhood_w = [neighborhood_w[_M]]
            neighborhood_b = np.sort(neighborhood_b, axis = 0)
            neighborhood_b = [neighborhood_b[_M]]
        neighborhood_w = np.mean(neighborhood_w, axis = 0)
        neighborhood_b = np.mean(neighborhood_b, axis = 0)
    #        print(neighborhood.shape)
        ave_w.append(neighborhood_w)
        ave_b.append(neighborhood_b)

    for node, w, b in zip(W, ave_w, ave_b):
        node.assign([w, b], sess) 

def node_update(W, data, sess, stepsize=1e-3):        
    for model, sample, label in zip(W, data.dist_data, data.dist_label):
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
        


if __name__ == "__main__":
    para = experiment_parameters(agents=20, dataset='MNIST', localsize_N=2000, iteration=100,
                                   screen=True, b=0, Byzantine='random', stepsize = 1e-1)
    #Generate the graph
    W_0, graph = gen_graph(para.M, para.con_rate, para.b)    
    local_set, test_data, test_label = data_prep(para.dataset, para.M, para.N, one_hot=True)
    neighbors = get_neighbor(graph)
    #Initialization
    tf.reset_default_graph()
    w_nodes = [linear_classifier(stepsize = para.stepsize) for node in range(para.M)]    
    sess = initialization()
    rec = []
 
    for iteration in range(para.T):
        print(iteration)
        #Communication 
        communication(w_nodes, neighbors, sess, para.b, para.screen)
        
#         if iteration%10 == 9:            
        #test over all test data
        accuracy = [acc_test(node, test_data, test_label) for node in w_nodes]
#             print(accuracy)
        rec.append(np.mean(accuracy))
        #node update using GD   
        node_update(w_nodes, local_set, sess, stepsize=para.stepsize/(iteration+1))

    
    sess.close()
    with open('./result/result_median_faultless.pickle', 'wb') as handle:
        pickle.dump(rec, handle)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
# Decentralized learning with krum

# Specify M nodes, N local samples, dataset, failure type, screening type, stepsize, connection rate, T iterations,
# b Byzantine agents

import numpy as np
from pre_proc import data_prep
from linear_classifier import linear_classifier
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
    def __init__(self, agents, dataset, localsize_N, iteration, stepsize=1e-4, 
                 screen=False, b=0, Byzantine='random', connection_rate=50):
        self.dataset = dataset
        self.M = agents
        self.T = iteration
        self.screen = screen
        self.b = b
        self.stepsize = stepsize
        self.N = localsize_N
        self.Byzantine = Byzantine
        self.con_rate = connection_rate

def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
    
    re = 1                      # regenerate if graph assumption not satisfied
    while re:
        graph = []
        for _ in range(nodes):
            graph.append([])
        for row in range(nodes):
            graph[row].append(1)
            for col in range(row + 1, nodes):
                d = random.randint(1, 100)
                if d < con_rate:
                    graph[row].append(1)     #form symmetric matrix row by row
                    graph[col].append(1)
                else:
                    graph[row].append(0)
                    graph[col].append(0)
        d_max = 0
        for row in graph:
            if sum(row) > d_max:
                d_max = sum(row)
        w = [row[:] for row in graph]
        for ind, row in enumerate(w):
            d = sum(row)
            w[ind] = [col/d_max for col in row]
            w[ind][ind] = 1 - (d - 1) / d_max
        if all([sum(row) >= 2 * b + 1 for row in graph]):
            re = 0
    return w, graph   

def get_neighbor(G):
    neighbor_list = []
    for node in G:
        neighbor = []
        for ind, n in enumerate(node):
            if n == 1:
                neighbor.append(ind)
        neighbor_list.append(neighbor)
    return neighbor_list

def one_hot(label):
    l_oh = []
    for i in label:
        new_l = [0] * 10
        new_l[i] = 1
        l_oh.append(new_l)
    return l_oh


def initialization():
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    return sess


def screen(grad, b, method):
    screened = Byzantine_algs[method](grad, b)
    return screened
    
def acc_test(model, t_data, t_label):
    acc = model.accuracy.eval(feed_dict={
            model.x:t_data, model.y_: t_label,})
    return acc
    
def communication(W, neighbor, sess, b=0, screen=False):
    wb = [node.weights() for node in W]
    # Byznatine nodes
    for node in range(b):
        wb[node][0] = np.random.rand(784,10) * 20
        wb[node][1] = np.random.rand(10) * 20
    ave_w = []
    ave_b = []
    for neighbor_list in neighbor:
        neighborhood_w = [wb[n][0] for n in neighbor_list]
        neighborhood_b = [wb[n][1] for n in neighbor_list] 
        if screen:
            #krum screening
            score = []
            for _g in neighborhood_w:
                dist = [np.linalg.norm(np.array(_g) - np.array(other)) for other in neighborhood_w]
                dist.sort()
                score.append(np.sum(dist[: (len(neighborhood_w) - b - 2)]))
            ind = score.index(min(score))
            neighborhood_w = [neighborhood_w[ind]]
            neighborhood_b = [neighborhood_b[ind]]            

        neighborhood_w = np.mean(neighborhood_w, axis = 0)
        neighborhood_b = np.mean(neighborhood_b, axis = 0)
    #        print(neighborhood.shape)
        ave_w.append(neighborhood_w)
        ave_b.append(neighborhood_b)

    for node, w, b in zip(W, ave_w, ave_b):
        node.assign([w, b], sess) 

def node_update(W, data, sess, stepsize=1e-3):        
    for model, sample, label in zip(W, data.dist_data, data.dist_label):
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
        


if __name__ == "__main__":
    for ep in range(20, 40):
        para = experiment_parameters(agents=20, dataset='MNIST', localsize_N=2000, iteration=200,
                                       screen=True, b=2, Byzantine='random', stepsize = 5e-2)
        #Generate the graph
        W_0, graph = gen_graph(para.M, para.con_rate, para.b)    
        local_set, test_data, test_label = data_prep(para.dataset, para.M, para.N, one_hot=True)
        neighbors = get_neighbor(graph)
        #Initialization
        tf.reset_default_graph()
        w_nodes = [linear_classifier(stepsize = para.stepsize) for node in range(para.M)]    
        sess = initialization()
        rec = []

        for iteration in range(para.T):
            print(iteration)
            #Communication 
            communication(w_nodes, neighbors, sess, para.b, para.screen)


            #test over all test data
            accuracy = [acc_test(node, test_data, test_label) for node in w_nodes[para.b:]]
    #             print(accuracy)
            rec.append(np.mean(accuracy))
            with open('./result/result_krum_b2_s%d.pickle'%ep, 'wb') as handle:
                pickle.dump(rec, handle)
            print(accuracy)
            #node update using GD   
            node_update(w_nodes, local_set, sess, stepsize=para.stepsize * (iteration + 1))


        sess.close()
        #krum b2 dec
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0
[0.1464, 0.1057, 0.1099, 0.1279, 0.1057, 0.0741, 0.0882, 0.1099, 0.0741, 0.0924, 0.1279, 0.1464, 0.0924, 0.1279, 0.1099, 0.1464, 0.0924, 0.1099]
1
[0.1431, 0.32, 0.32, 0.3214, 0.098, 0.3666, 0.3666, 0.32, 0.098, 0.1405, 0.098, 0.32, 0.1405, 0.32, 0.3666, 0.3666, 0.3666, 0.32]
2
[0.3467, 0.2957, 0.1439, 0.2118, 0.3467, 0.1875, 0.1875, 0.2957, 0.3485, 0.2957, 0.2957, 0.3467, 0.4709, 0.2118, 0.2118, 0.2957, 0.1111, 0.2957]
3
[0.2616, 0.2928, 0.097, 0.2828, 0.2828, 0.2616, 0.2504, 0.2928, 0.2616, 0.3064, 0.2928, 0.2828, 0.3425, 0.2616, 0.2616, 0.2928, 0.2504, 0.2928]
4
[0.301, 0.3071, 0.3361, 0.3361, 0.2704, 0.3216, 0.3121, 0.3361, 0.301, 0.2704, 0.3071, 0.2811, 0.5577, 0.3121, 0.301, 0.3071, 0.1306, 0.2811]
5
[0.2183, 0.254, 0.2

[0.7892, 0.7892, 0.7892, 0.7892, 0.6666, 0.6666, 0.6666, 0.7892, 0.6709, 0.6709, 0.7892, 0.6709, 0.6709, 0.7892, 0.7892, 0.7892, 0.6709, 0.7892]
53
[0.7892, 0.7892, 0.7892, 0.7892, 0.6617, 0.6617, 0.6617, 0.7892, 0.6665, 0.6665, 0.7892, 0.6665, 0.6665, 0.7892, 0.7892, 0.7892, 0.6665, 0.7892]
54
[0.7892, 0.7892, 0.7892, 0.7892, 0.6564, 0.6564, 0.6564, 0.7892, 0.6604, 0.6564, 0.7892, 0.6604, 0.6604, 0.7892, 0.7892, 0.7892, 0.6604, 0.7892]
55
[0.7892, 0.7892, 0.7892, 0.7892, 0.653, 0.653, 0.653, 0.7892, 0.6546, 0.6546, 0.7892, 0.6546, 0.6546, 0.7892, 0.7892, 0.7892, 0.6546, 0.7892]
56
[0.7892, 0.7892, 0.7892, 0.7892, 0.6486, 0.6486, 0.6486, 0.7892, 0.6481, 0.6486, 0.7892, 0.6481, 0.6481, 0.7892, 0.7892, 0.7892, 0.6481, 0.7892]
57
[0.7892, 0.7892, 0.7892, 0.7892, 0.6451, 0.6451, 0.6451, 0.7892, 0.6429, 0.6429, 0.7892, 0.6429, 0.6429, 0.7892, 0.7892, 0.7892, 0.6429, 0.7892]
58
[0.7892, 0.7892, 0.7892, 0.7892, 0.6416, 0.6416, 0.6416, 0.7892, 0.6369, 0.6416, 0.7892, 0.6369, 0.6369, 0.7892, 0.

[0.7892, 0.7892, 0.7892, 0.7892, 0.4942, 0.4942, 0.4942, 0.7892, 0.357, 0.357, 0.7892, 0.357, 0.357, 0.7892, 0.7892, 0.7892, 0.0761, 0.7892]
109
[0.7892, 0.7892, 0.7892, 0.7892, 0.4924, 0.4924, 0.4924, 0.7892, 0.3545, 0.3545, 0.7892, 0.3545, 0.3545, 0.7892, 0.7892, 0.7892, 0.0931, 0.7892]
110
[0.7892, 0.7892, 0.7892, 0.7892, 0.4896, 0.4896, 0.4896, 0.7892, 0.3517, 0.3517, 0.7892, 0.3517, 0.3517, 0.7892, 0.7892, 0.7892, 0.0621, 0.7892]
111
[0.7892, 0.7892, 0.7892, 0.7892, 0.4876, 0.4876, 0.4876, 0.7892, 0.3488, 0.3488, 0.7892, 0.3488, 0.3488, 0.7892, 0.7892, 0.7892, 0.0807, 0.7892]
112
[0.7892, 0.7892, 0.7892, 0.7892, 0.4853, 0.4853, 0.4853, 0.7892, 0.3469, 0.3469, 0.7892, 0.3469, 0.3469, 0.7892, 0.7892, 0.7892, 0.0413, 0.7892]
113
[0.7892, 0.7892, 0.7892, 0.7892, 0.4829, 0.4829, 0.4829, 0.7892, 0.3448, 0.3448, 0.7892, 0.3448, 0.3448, 0.7892, 0.7892, 0.7892, 0.1628, 0.7892]
114
[0.7892, 0.7892, 0.7892, 0.7892, 0.4807, 0.4807, 0.4807, 0.7892, 0.343, 0.343, 0.7892, 0.343, 0.343, 0.7892, 0

[0.7892, 0.7892, 0.7892, 0.7892, 0.3783, 0.3783, 0.3783, 0.7892, 0.2829, 0.2829, 0.7892, 0.2829, 0.2829, 0.7892, 0.7892, 0.7892, 0.1686, 0.7892]
165
[0.7892, 0.7892, 0.7892, 0.7892, 0.376, 0.376, 0.376, 0.7892, 0.2825, 0.2825, 0.7892, 0.2825, 0.2825, 0.7892, 0.7892, 0.7892, 0.1326, 0.7892]
166
[0.7892, 0.7892, 0.7892, 0.7892, 0.3746, 0.3746, 0.3746, 0.7892, 0.2821, 0.2821, 0.7892, 0.2821, 0.2821, 0.7892, 0.7892, 0.7892, 0.0817, 0.7892]
167
[0.7892, 0.7892, 0.7892, 0.7892, 0.3739, 0.3739, 0.3739, 0.7892, 0.2807, 0.2807, 0.7892, 0.2807, 0.2807, 0.7892, 0.7892, 0.7892, 0.1247, 0.7892]
168
[0.7892, 0.7892, 0.7892, 0.7892, 0.3715, 0.3715, 0.3715, 0.7892, 0.2802, 0.2802, 0.7892, 0.2802, 0.2802, 0.7892, 0.7892, 0.7892, 0.204, 0.7892]
169
[0.7892, 0.7892, 0.7892, 0.7892, 0.3698, 0.3698, 0.3698, 0.7892, 0.2798, 0.2798, 0.7892, 0.2798, 0.2798, 0.7892, 0.7892, 0.7892, 0.0666, 0.7892]
170
[0.7892, 0.7892, 0.7892, 0.7892, 0.3684, 0.3684, 0.3684, 0.7892, 0.2791, 0.2791, 0.7892, 0.2791, 0.2791, 0.789

[0.6331, 0.6331, 0.6331, 0.6331, 0.4814, 0.6331, 0.6331, 0.5176, 0.6331, 0.6331, 0.5688, 0.6331, 0.5176, 0.6331, 0.6331, 0.6331, 0.6331, 0.5541]
21
[0.6287, 0.5531, 0.6928, 0.6287, 0.6216, 0.6287, 0.6287, 0.5328, 0.5289, 0.6287, 0.6287, 0.6928, 0.5912, 0.6287, 0.6287, 0.6287, 0.6287, 0.5912]
22
[0.6432, 0.4569, 0.4569, 0.6432, 0.5196, 0.6076, 0.6432, 0.6432, 0.5548, 0.5196, 0.6432, 0.6355, 0.6076, 0.6076, 0.6076, 0.6076, 0.6432, 0.6432]
23
[0.5707, 0.5645, 0.5707, 0.5463, 0.393, 0.5707, 0.5707, 0.5645, 0.5707, 0.5707, 0.5645, 0.5707, 0.5645, 0.5463, 0.5707, 0.5039, 0.5707, 0.5645]
24
[0.5399, 0.5399, 0.5339, 0.5339, 0.5546, 0.5399, 0.5339, 0.5399, 0.5809, 0.5838, 0.5399, 0.5809, 0.5399, 0.5339, 0.5339, 0.5339, 0.5809, 0.5399]
25
[0.5948, 0.5396, 0.6302, 0.5948, 0.5041, 0.5948, 0.5948, 0.6871, 0.5838, 0.6302, 0.5576, 0.5838, 0.6871, 0.5396, 0.5396, 0.5396, 0.6302, 0.6302]
26
[0.5252, 0.5478, 0.5252, 0.5252, 0.4845, 0.5252, 0.5252, 0.5478, 0.6732, 0.5252, 0.5252, 0.6732, 0.5252, 0.5252, 

...

[0.5068, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.5527, 0.8118, 0.6129, 0.3909, 0.6129, 0.8118, 0.5527, 0.5527, 0.5068, 0.5527, 0.3909, 0.8118]
101
[0.5054, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.5492, 0.8118, 0.6104, 0.3866, 0.6104, 0.8118, 0.5492, 0.5492, 0.5054, 0.5492, 0.3866, 0.8118]
102
[0.5046, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.5462, 0.8118, 0.6077, 0.3826, 0.6077, 0.8118, 0.5462, 0.5462, 0.5046, 0.5462, 0.3826, 0.8118]
103
[0.6322, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.542, 0.8118, 0.6054, 0.3783, 0.6054, 0.8118, 0.542, 0.542, 0.6054, 0.542, 0.5034, 0.8118]
104
[0.6298, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.5397, 0.8118, 0.6037, 0.3736, 0.6037, 0.8118, 0.5397, 0.5397, 0.6037, 0.5397, 0.3736, 0.8118]
105
[0.6271, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.5365, 0.8118, 0.6013, 0.3694, 0.6013, 0.8118, 0.5365, 0.5365, 0.6013, 0.5365, 0.3694, 0.8118]
106
[0.6246, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.5321, 0.8118, 0.5997, 0.3648, 0.5997, 0.8118, 0.5321, 0.532

[0.4875, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.3859, 0.8118, 0.4828, 0.2111, 0.4828, 0.8118, 0.3859, 0.3859, 0.4828, 0.3859, 0.2111, 0.8118]
157
[0.485, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.3841, 0.8118, 0.4809, 0.2098, 0.4809, 0.8118, 0.3841, 0.3841, 0.4809, 0.3841, 0.4538, 0.8118]
158
[0.4815, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.383, 0.8118, 0.4793, 0.2086, 0.4793, 0.8118, 0.383, 0.383, 0.4793, 0.383, 0.2086, 0.8118]
159
[0.4782, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.3813, 0.8118, 0.477, 0.2063, 0.477, 0.8118, 0.3813, 0.3813, 0.477, 0.3813, 0.4523, 0.8118]
160
[0.4745, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.3803, 0.8118, 0.4756, 0.2046, 0.4756, 0.8118, 0.3803, 0.3803, 0.4756, 0.3803, 0.2046, 0.8118]
161
[0.4716, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.3786, 0.8118, 0.4737, 0.2031, 0.4737, 0.8118, 0.3786, 0.3786, 0.4737, 0.3786, 0.4499, 0.8118]
162
[0.4683, 0.8118, 0.8118, 0.8118, 0.8118, 0.8118, 0.3772, 0.8118, 0.4718, 0.201, 0.4718, 0.8118, 0.3772, 0.3772, 0.

[0.4402, 0.3926, 0.3929, 0.4613, 0.3948, 0.4402, 0.4613, 0.3948, 0.3948, 0.3926, 0.4613, 0.43, 0.3926, 0.3948, 0.3948, 0.4162, 0.3948, 0.3926]
13
[0.4202, 0.3273, 0.4202, 0.3705, 0.4202, 0.4202, 0.3273, 0.4202, 0.4202, 0.3705, 0.4669, 0.4072, 0.3705, 0.3705, 0.5331, 0.4669, 0.4669, 0.4202]
14
[0.5061, 0.3329, 0.4416, 0.5061, 0.5061, 0.5175, 0.5061, 0.5061, 0.3712, 0.464, 0.5175, 0.5175, 0.3615, 0.5061, 0.5061, 0.5175, 0.5175, 0.5061]
15
[0.6673, 0.6673, 0.6673, 0.6673, 0.5928, 0.6673, 0.5928, 0.6673, 0.6673, 0.6673, 0.5928, 0.6673, 0.6673, 0.6673, 0.5928, 0.5754, 0.5604, 0.6673]
16
[0.6295, 0.6966, 0.6734, 0.6295, 0.6218, 0.6295, 0.6295, 0.6966, 0.6734, 0.6966, 0.6218, 0.6295, 0.6966, 0.6966, 0.6966, 0.5778, 0.6734, 0.6966]
17
[0.6623, 0.7331, 0.5887, 0.6623, 0.5002, 0.6623, 0.6761, 0.6623, 0.6623, 0.6761, 0.7377, 0.5399, 0.7331, 0.6623, 0.7331, 0.6385, 0.7054, 0.6623]
18
[0.6887, 0.6748, 0.6742, 0.6887, 0.6887, 0.5749, 0.6887, 0.6887, 0.5749, 0.6887, 0.6887, 0.6887, 0.6748, 0.6887, 0.

[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.6764, 0.8056, 0.8056, 0.6764, 0.8056, 0.7557, 0.7557, 0.8056, 0.7557, 0.8056, 0.6644, 0.7557, 0.8056]
69
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.6722, 0.8056, 0.8056, 0.6722, 0.8056, 0.7533, 0.7533, 0.8056, 0.7533, 0.8056, 0.6582, 0.7533, 0.8056]
70
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.6685, 0.8056, 0.8056, 0.6685, 0.8056, 0.7515, 0.7515, 0.8056, 0.7515, 0.8056, 0.6537, 0.7515, 0.8056]
71
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.6648, 0.8056, 0.8056, 0.6648, 0.8056, 0.7503, 0.7503, 0.8056, 0.7503, 0.8056, 0.6485, 0.7503, 0.8056]
72
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.6601, 0.8056, 0.8056, 0.6601, 0.8056, 0.7479, 0.7479, 0.8056, 0.7479, 0.8056, 0.6424, 0.7479, 0.8056]
73
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.6564, 0.8056, 0.8056, 0.6564, 0.8056, 0.7458, 0.7458, 0.8056, 0.7458, 0.8056, 0.6365, 0.7458, 0.8056]
74
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.5552, 0.8056, 0.8056, 0.6525, 0.8056, 0.7431, 0.7431, 0.8056, 0.7431,

[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.4484, 0.8056, 0.8056, 0.4484, 0.8056, 0.3014, 0.6122, 0.8056, 0.6122, 0.8056, 0.3006, 0.6122, 0.8056]
125
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.4446, 0.8056, 0.8056, 0.4446, 0.8056, 0.298, 0.6101, 0.8056, 0.6101, 0.8056, 0.2987, 0.6101, 0.8056]
126
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.4415, 0.8056, 0.8056, 0.4415, 0.8056, 0.2951, 0.6077, 0.8056, 0.6077, 0.8056, 0.2968, 0.6077, 0.8056]
127
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.4381, 0.8056, 0.8056, 0.4381, 0.8056, 0.2919, 0.6059, 0.8056, 0.6059, 0.8056, 0.2946, 0.6059, 0.8056]
128
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.4344, 0.8056, 0.8056, 0.4344, 0.8056, 0.2887, 0.6035, 0.8056, 0.6035, 0.8056, 0.2923, 0.6035, 0.8056]
129
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.431, 0.8056, 0.8056, 0.431, 0.8056, 0.2856, 0.6018, 0.8056, 0.6018, 0.8056, 0.2901, 0.6018, 0.8056]
130
[0.8056, 0.8056, 0.8056, 0.8056, 0.8056, 0.4278, 0.8056, 0.8056, 0.4278, 0.8056, 0.2817, 0.5995, 0.8056, 0.59

In [1]:
# Decentralized learning with bulyan

# Specify M nodes, N local samples, dataset, failure type, screening type, stepsize, connection rate, T iterations,
# b Byzantine agents

import numpy as np
from pre_proc import data_prep
from linear_classifier import linear_classifier
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
    def __init__(self, agents, dataset, localsize_N, iteration, stepsize=1e-4, 
                 screen=False, b=0, Byzantine='random', connection_rate=50):
        self.dataset = dataset
        self.M = agents
        self.T = iteration
        self.screen = screen
        self.b = b
        self.stepsize = stepsize
        self.N = localsize_N
        self.Byzantine = Byzantine
        self.con_rate = connection_rate

def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
    
    re = 1                      # regenerate if graph assumption not satisfied
    while re:
        graph = []
        for _ in range(nodes):
            graph.append([])
        for row in range(nodes):
            graph[row].append(1)
            for col in range(row + 1, nodes):
                d = random.randint(1, 100)
                if d < con_rate:
                    graph[row].append(1)     #form symmetric matrix row by row
                    graph[col].append(1)
                else:
                    graph[row].append(0)
                    graph[col].append(0)
        d_max = 0
        for row in graph:
            if sum(row) > d_max:
                d_max = sum(row)
        w = [row[:] for row in graph]
        for ind, row in enumerate(w):
            d = sum(row)
            w[ind] = [col/d_max for col in row]
            w[ind][ind] = 1 - (d - 1) / d_max
        if all([sum(row) >= 4 * b + 3 for row in graph]):
            re = 0
    return w, graph   

def get_neighbor(G):
    neighbor_list = []
    for node in G:
        neighbor = []
        for ind, n in enumerate(node):
            if n == 1:
                neighbor.append(ind)
        neighbor_list.append(neighbor)
    return neighbor_list

def one_hot(label):
    l_oh = []
    for i in label:
        new_l = [0] * 10
        new_l[i] = 1
        l_oh.append(new_l)
    return l_oh


def initialization():
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    return sess


def screen(grad, b, method):
    screened = Byzantine_algs[method](grad, b)
    return screened
    
def acc_test(model, t_data, t_label):
    acc = model.accuracy.eval(feed_dict={
            model.x:t_data, model.y_: t_label,})
    return acc
    
def communication(W, neighbor, sess, b=0, screen=False):
    wb = [node.weights() for node in W]
    # Byznatine nodes
#     for node in range(b):
#         wb[node][0] = np.random.rand(784,10) * 20
#         wb[node][1] = np.random.rand(10) * 20
    ave_w = []
    ave_b = []
    for neighbor_list in neighbor:
        neighborhood_w = [wb[n][0] for n in neighbor_list]
        neighborhood_b = [wb[n][1] for n in neighbor_list] 


        if screen:
            bulyan_w = []
            bulyan_b = []
            M = len(neighborhood_w)
            #bulyan screening
            for m in range(M-2*b):
                score = []
                for _g in neighborhood_w:
                    dist = [np.linalg.norm(np.array(_g) - np.array(other)) for other in neighborhood_w]
                    dist.sort()
                    score.append(np.sum(dist[: (len(neighborhood_w) - b - 2)]))
                ind = score.index(min(score))
                bulyan_w.append(neighborhood_w[ind])
                bulyan_b.append(neighborhood_b[ind])
                del neighborhood_w[ind]
                del neighborhood_b[ind]
            bulyan_w = np.sort(bulyan_w, axis = 0)
            bulyan_b = np.sort(bulyan_b, axis = 0)
            neighborhood_w = bulyan_w[b : -b]
            neighborhood_b = bulyan_b[b : -b]
               
        neighborhood_w = np.mean(neighborhood_w, axis = 0)
        neighborhood_b = np.mean(neighborhood_b, axis = 0)
    #        print(neighborhood.shape)
        ave_w.append(neighborhood_w)
        ave_b.append(neighborhood_b)

    for node, w, b in zip(W, ave_w, ave_b):
        node.assign([w, b], sess) 

def node_update(W, data, sess, stepsize=1e-3):        
    for model, sample, label in zip(W, data.dist_data, data.dist_label):
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
        


if __name__ == "__main__":
    for ep in range(10):
        para = experiment_parameters(agents=20, dataset='MNIST', localsize_N=2000, iteration=1000,
                                       screen=True, b=2, Byzantine='random', stepsize = 1e-1, connection_rate=60)
        #Generate the graph
        W_0, graph = gen_graph(para.M, para.con_rate, para.b)    
        local_set, test_data, test_label = data_prep(para.dataset, para.M, para.N, one_hot=True)
        neighbors = get_neighbor(graph)
        #Initialization
        tf.reset_default_graph()
        w_nodes = [linear_classifier(stepsize = para.stepsize) for node in range(para.M)]    
        sess = initialization()
        rec = []

        for iteration in range(para.T):
            print(iteration)
            #Communication 
            communication(w_nodes, neighbors, sess, para.b, para.screen)


            #test over all test data
            accuracy = [acc_test(node, test_data, test_label) for node in w_nodes[para.b:]]
    #             print(accuracy)
            rec.append(np.mean(accuracy))
            with open('./result/result_bulyan_faultless_1%d.pickle'%ep, 'wb') as handle:
                pickle.dump(rec, handle)
            print(accuracy)
            #node update using GD   
            node_update(w_nodes, local_set, sess, stepsize=para.stepsize/(1))


        sess.close()
    
#Bulyan_faultless



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0
[0.042, 0.0963, 0.094, 0.0698, 0.0693, 0.1108, 0.0907, 0.0908, 0.0661, 0.07, 0.0916, 0.0768, 0.0701, 0.0718, 0.1219, 0.0957, 0.1048, 0.107]
1
[0.1115, 0.1109, 0.1209, 0.1421, 0.2535, 0.2655, 0.1304, 0.1184, 0.1649, 0.1305, 0.1952, 0.2342, 0.1827, 0.2037, 0.1579, 0.1025, 0.205, 0.2547]
2
[0.6329, 0.4564, 0.5438, 0.5238, 0.5441, 0.5312, 0.5044, 0.5296, 0.5506, 0.4711, 0.4898, 0.5565, 0.5005, 0.3915, 0.4974, 0.6239, 0.5375, 0.4904]
3
[0.4431, 0.3786, 0.4127, 0.4082, 0.3504, 0.3718, 0.4245, 0.4086, 0.3997, 0.3695, 0.3779, 0.4304, 0.3634, 0.3683, 0.3849, 0.4097, 0.4047, 0.4029]
4
[0.4113, 0.4334, 0.4473, 0.4175, 0.4347, 0.4038, 0.4333, 0.4316, 0.4252, 0.4175, 0.4334, 0.4098, 0.4207, 0.4115, 0.4294, 0.4335, 0.4106, 0.4116]
5
[0.480

[0.8674, 0.8604, 0.8646, 0.8684, 0.8695, 0.8651, 0.8668, 0.8631, 0.8679, 0.8695, 0.8683, 0.8663, 0.8686, 0.8625, 0.8662, 0.8629, 0.8634, 0.8693]
53
[0.8671, 0.8673, 0.8654, 0.8688, 0.8697, 0.8658, 0.8683, 0.8659, 0.8693, 0.8685, 0.8678, 0.8675, 0.868, 0.8648, 0.8654, 0.8641, 0.8645, 0.8687]
54
[0.866, 0.8668, 0.8662, 0.8698, 0.8678, 0.8664, 0.8634, 0.8659, 0.8639, 0.8673, 0.8676, 0.861, 0.8683, 0.8665, 0.864, 0.8642, 0.8616, 0.867]
55
[0.8691, 0.8674, 0.8693, 0.87, 0.8674, 0.8655, 0.8694, 0.8651, 0.8682, 0.8679, 0.868, 0.8686, 0.8673, 0.8648, 0.8672, 0.8641, 0.8669, 0.8676]
56
[0.8657, 0.8694, 0.8699, 0.87, 0.8703, 0.8669, 0.8629, 0.8697, 0.8702, 0.8711, 0.8696, 0.8661, 0.8697, 0.869, 0.868, 0.8682, 0.8637, 0.8693]
57
[0.8657, 0.8633, 0.8644, 0.8654, 0.8653, 0.8655, 0.8661, 0.8652, 0.8679, 0.8678, 0.8679, 0.8697, 0.8661, 0.8631, 0.8647, 0.861, 0.8636, 0.8709]
58
[0.8698, 0.866, 0.87, 0.8721, 0.8693, 0.8667, 0.87, 0.8677, 0.8699, 0.8712, 0.8694, 0.8669, 0.8697, 0.8678, 0.8694, 0.8679, 0

[0.8684, 0.8674, 0.8702, 0.87, 0.8679, 0.8654, 0.8705, 0.8667, 0.8682, 0.8669, 0.867, 0.8619, 0.8698, 0.8676, 0.8665, 0.8648, 0.8628, 0.8659]
110
[0.8658, 0.8654, 0.8561, 0.8675, 0.8633, 0.8639, 0.8656, 0.8672, 0.8648, 0.8675, 0.8655, 0.8652, 0.8693, 0.8637, 0.8655, 0.8606, 0.8651, 0.8673]
111
[0.8691, 0.8652, 0.8694, 0.8703, 0.8662, 0.8669, 0.8676, 0.8664, 0.8699, 0.8703, 0.871, 0.8647, 0.8713, 0.8658, 0.8665, 0.8649, 0.8623, 0.8695]
112
[0.8642, 0.8666, 0.8617, 0.863, 0.8637, 0.8649, 0.8605, 0.8667, 0.8621, 0.865, 0.8647, 0.864, 0.8661, 0.8644, 0.8634, 0.8596, 0.8629, 0.8654]
113
[0.8703, 0.8671, 0.8689, 0.8691, 0.866, 0.8665, 0.8659, 0.8683, 0.8671, 0.8697, 0.8688, 0.8673, 0.8719, 0.8681, 0.8663, 0.8636, 0.8628, 0.8663]
114
[0.8673, 0.8676, 0.8708, 0.8652, 0.8655, 0.8666, 0.8635, 0.8624, 0.8648, 0.8688, 0.8666, 0.8622, 0.8671, 0.865, 0.8655, 0.863, 0.8608, 0.8631]
115
[0.8672, 0.8684, 0.8705, 0.8708, 0.867, 0.8636, 0.863, 0.8645, 0.8696, 0.867, 0.8655, 0.8619, 0.8687, 0.8675, 0.865,

[0.8605, 0.8555, 0.8567, 0.8511, 0.8585, 0.8585, 0.8475, 0.8567, 0.8525, 0.8584, 0.8594, 0.8615, 0.8556, 0.842, 0.8644, 0.8622, 0.8606, 0.861]
166
[0.8624, 0.8636, 0.8647, 0.8679, 0.8608, 0.8656, 0.8608, 0.8659, 0.8651, 0.8638, 0.8681, 0.8681, 0.8669, 0.8682, 0.8679, 0.8672, 0.8668, 0.8659]
167
[0.8595, 0.8609, 0.8534, 0.8542, 0.8576, 0.8604, 0.8576, 0.8602, 0.8568, 0.8646, 0.8623, 0.8685, 0.861, 0.8555, 0.8644, 0.8563, 0.8632, 0.8685]
168
[0.8664, 0.8666, 0.8631, 0.8675, 0.8685, 0.8691, 0.8609, 0.8698, 0.867, 0.8664, 0.8682, 0.8683, 0.8677, 0.8698, 0.869, 0.87, 0.8691, 0.8691]
169
[0.8678, 0.867, 0.8655, 0.8633, 0.8702, 0.8676, 0.861, 0.865, 0.8647, 0.8686, 0.8669, 0.869, 0.8685, 0.8588, 0.8678, 0.8663, 0.8664, 0.8692]
170
[0.8676, 0.8688, 0.8697, 0.8681, 0.8634, 0.8665, 0.8611, 0.8682, 0.8671, 0.8693, 0.8675, 0.8661, 0.8693, 0.8671, 0.8669, 0.8657, 0.8669, 0.8677]
171
[0.8672, 0.8659, 0.8682, 0.869, 0.869, 0.867, 0.8672, 0.8698, 0.868, 0.87, 0.867, 0.8705, 0.869, 0.8675, 0.8684, 0.86

[0.8643, 0.851, 0.8596, 0.8634, 0.8496, 0.8666, 0.8361, 0.8605, 0.8619, 0.856, 0.8639, 0.8689, 0.8631, 0.8643, 0.8639, 0.8618, 0.8682, 0.8647]
222
[0.8532, 0.842, 0.828, 0.8464, 0.8546, 0.86, 0.8359, 0.8408, 0.8419, 0.8491, 0.8586, 0.8656, 0.8519, 0.8532, 0.8539, 0.855, 0.8621, 0.864]
223
[0.866, 0.8584, 0.8647, 0.8653, 0.8536, 0.8661, 0.8586, 0.8645, 0.8646, 0.8639, 0.8666, 0.8654, 0.8667, 0.8671, 0.8631, 0.8637, 0.8653, 0.8642]
224
[0.862, 0.8615, 0.8501, 0.8616, 0.8652, 0.8657, 0.8623, 0.856, 0.8538, 0.865, 0.8637, 0.8638, 0.8628, 0.8619, 0.8606, 0.8623, 0.8623, 0.8666]
225
[0.8626, 0.862, 0.8661, 0.8652, 0.8561, 0.8661, 0.8511, 0.8664, 0.8648, 0.8614, 0.8667, 0.865, 0.8644, 0.8665, 0.8652, 0.8646, 0.8641, 0.8654]
226
[0.8669, 0.8661, 0.8593, 0.8605, 0.8653, 0.866, 0.8668, 0.8662, 0.8618, 0.867, 0.8663, 0.8659, 0.8666, 0.8649, 0.864, 0.866, 0.8658, 0.8674]
227
[0.8647, 0.8639, 0.868, 0.8648, 0.8594, 0.8661, 0.8635, 0.8667, 0.8668, 0.8654, 0.866, 0.8631, 0.8667, 0.8641, 0.8654, 0.863

...

752
[0.8616, 0.8612, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
753
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
754
[0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
755
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
756
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
757
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
758
[0.8616, 0.862, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8622, 0.8616, 0.8616

[0.8616, 0.8612, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
808
[0.8616, 0.8617, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0.8616, 0.8616, 0.8619, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
809
[0.8616, 0.8609, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8609, 0.8616, 0.8616, 0.8611, 0.8616, 0.8616, 0.8616, 0.8616]
810
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
811
[0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
812
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8614, 0.8616, 0.8616, 0.8616, 0.8616]
813
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0

[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616]
863
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616]
864
[0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616]
865
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616]
866
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616]
867
[0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8618, 0.8616, 0.8616, 0.8613, 0.8616, 0.8616, 0.8616, 0.8616]
868
[0.8616, 0.8616, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0.8616, 0.8616, 0.8617, 0.8616, 0.8616, 0

[0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
918
[0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
919
[0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
920
[0.8615, 0.8616, 0.8615, 0.8615, 0.8615, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8612, 0.8615, 0.8615, 0.8615, 0.8615]
921
[0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
922
[0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
923
[0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0

[0.8615, 0.8616, 0.8615, 0.8615, 0.8616, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
973
[0.8615, 0.8613, 0.8615, 0.8615, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
974
[0.8615, 0.8613, 0.8615, 0.8615, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
975
[0.8615, 0.8613, 0.8615, 0.8615, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
976
[0.8615, 0.8616, 0.8615, 0.8615, 0.8616, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
977
[0.8615, 0.8613, 0.8615, 0.8615, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0.8613, 0.8615, 0.8615, 0.8615, 0.8615]
978
[0.8615, 0.8613, 0.8615, 0.8615, 0.8616, 0.8615, 0.8615, 0.8615, 0.8615, 0.8615, 0.8618, 0.8615, 0.8615, 0

In [1]:
#  %%writefile dis_trimmed_mean_b4.py
# %%writefile Distributed_learning.py
# %load Distributed_learning.py
# Distributed learning

# Specify M agents, N local samples, dataset, failure type, screening type, stepsize, batchsize, T iterations,
# b Byzantine agents

import numpy as np
from pre_proc import data_prep
from CNN_model import CNN
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
    def __init__(self, agents, dataset, localsize_N, iteration, batchsize=0, stepsize=1, 
                 screen=False, b=0, Byzantine='random'):
        self.dataset = dataset
        self.M = agents
        self.T = iteration
        self.screen = screen
        self.b = b
        self.stepsize = stepsize
        self.batchsize = batchsize
        self.N = localsize_N
        self.Byzantine = Byzantine
        if self.batchsize >  self.N:
            raise ValueError("Batch size is large than local datasize")


def data_distribution(dataset, nodes_M, sample_N):#samples will be distributed evenly to each node
    distributed_data, test_data, test_label = data_prep(dataset, nodes_M, nodes_M * sample_N, one_hot=True)
    return distributed_data, test_data, test_label

def initialization(W):
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    weight = W[0].weights()
    for w in W:
        w.assign(weight, sess)
    return sess


def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
    
    re = 1                      # regenerate if graph assumption not satisfied
    while re:
        graph = []
        for _ in range(nodes):
            graph.append([])
        for row in range(nodes):
            graph[row].append(1)
            for col in range(row + 1, nodes):
                d = random.randint(1, 100)
                if d < con_rate:
                    graph[row].append(1)     #form symmetric matrix row by row
                    graph[col].append(1)
                else:
                    graph[row].append(0)
                    graph[col].append(0)
        d_max = 0
        for row in graph:
            if sum(row) > d_max:
                d_max = sum(row)
        w = [row[:] for row in graph]
        for ind, row in enumerate(w):
            d = sum(row)
            w[ind] = [col/d_max for col in row]
            w[ind][ind] = 1 - (d - 1) / d_max
        if all([sum(row) >= 2 * b + 1 for row in graph]):
            re = 0
    return w, graph

def get_neighbor(G):
    neighbor_list = []
    for node in G:
        neighbor = []
        for ind, n in enumerate(node):
            if n == 1:
                neighbor.append(ind)
        neighbor_list.append(neighbor)
    return neighbor_list

def Byz_nonfaulty(W, b, data, sess, scaling=1):
    for model, sample, label in zip(W[:b], data.dist_data[:b], data.dist_label[:b]):        
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.keep_prob : 0.5})
    return W[:b]        


def Byz_random(W, b, data, sess, scaling=1):
    for model in W[:b]:
        weight = model.weights()
        new = [np.random.rand(*layer.shape) * scaling for layer in weight]
        model.assign(new, sess)
    return W[:b]

def Byz_flip(W, b, data, sess, scaling=1):
    Byz_data = data.dist_data[:b] * (-1) * scaling
    for model, sample, label in zip(W[:b], Byz_data, data.dist_label[:b]):        
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.keep_prob : 0.5})
    return W[:b]  

def Byz_omniscient(W, b, data, sess, scaling=1):
    all_weights = [node.weights() for node in W[b:]]
    ave = []
    for layer in len(all_weights[b]):
        one_layer = [node[layer] for node in all_weights]
        one_layer = np.mean(one_layer, axis=0)           
        ave.append(one_layer)
    for node in W[:b]:
        node.assign(ave, sess)
    return W[:b]

def Byzantine(W, b, data, sess, strategy='nonfaulty', scaling=1):
    strategies = {'nonfaulty': Byz_nonfaulty, 'random': Byz_random, 'flip': Byz_flip, 'omniscient': Byz_omniscient}    
    w_B = strategies[strategy](W, b, data, sess)
    return w_B

    
def acc_test(model, t_data, t_label):
    acc = model.accuracy.eval(feed_dict={
            model.x:t_data, model.y_: t_label, model.keep_prob: 1.0})
    return acc
    
def local_update(W, sess, data, b):
    for model, sample, label in zip(W[b:], data.dist_data[b:], data.dist_label[b:]):        
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.keep_prob : 0.5})
    return W

def Trimmed_mean(W, neighbor, b):
    new_w = []
    for node in range(len(W)):
        ave = []
        for layer in range(len(W[node])):
            neighborhood = []
            for n in neighbor[node]:
                neighborhood.append(W[n][layer])
            neighborhood = np.sort(neighborhood, axis = 0)
            neighborhood = neighborhood[b : -b]
            neighborhood = np.mean(neighborhood, axis = 0) 
#        print(neighborhood.shape)
            ave.append(neighborhood)
        new_w.append(ave)   
    return new_w

def Krum(W, neighbor, b):
    new_w = []   
    #score is calculated for only the last layer
    for node in range(len(W)):
        score = []
        neighborhood = []
        neighbor_list = []
        for n in neighbor[node]:
            neighborhood.append(W[n][-1])
            neighbor_list.append(n)
        for g in neighborhood:
            dist = [np.abs(other - g) for other in neighborhood]
            dist = np.sort(dist)
            score.append(np.sum(dist[:(len(neighborhood) - b - 2)]))
        ind = score.index(min(score))
        new_w.append(W[neighbor_list[ind]]) 
    return new_w    



def Median(W, neighbor, b):
    new_w = []
    for node in range(len(W)):
        ave = []
        for layer in range(len(W[node])):
            neighborhood = []
            for n in neighbor[node]:
                neighborhood.append(W[n][layer])
            neighborhood = np.median(neighborhood, axis = 0)
#        print(neighborhood.shape)
            ave.append(neighborhood)
        new_w.append(ave)   
    return new_w


def Bulyan(W, neighbor, b):
    new_w = []   
    #score is calculated for only the last layer
    for node in range(len(W)):
        neighborhood = []
        neighbor_list = []
        for n in neighbor[node]:
            neighborhood.append(W[n][-1])
            neighbor_list.append(n)
        M = len(neighbor_list)
        second_round = []
        for _ in range(M - 2 * b):
            score = []
            for g in neighborhood:
                dist = [np.abs(other - g) for other in neighborhood]
                dist = np.sort(dist)
                score.append(np.sum(dist[:(len(neighborhood) - b - 2)]))
            ind = score.index(min(score))
            second_round.append(W[neighbor_list[ind]])
            del neighborhood[ind]
            del neighbor_list[ind]
        ave = []
        for layer in range(len(second_round[0])):
            current_layer = [node[layer] for node in second_round]
            current_layer = np.sort(current_layer, axis = 0)
            current_layer = current_layer[b : -b]
            current_layer = np.mean(current_layer, axis = 0)
            ave.append(current_layer)
        new_w.append(ave)          
    return new_w

def screening(W, neighbor, b, method='Trim'):
    method_list = {'Trim': Trimmed_mean, 'Median': Median, 'Krum': Krum, 'Bulyan': Bulyan}
    aggregated = method_list[method](W, neighbor, b)
    return aggregated


def BRIDGE_communication(W, neighbor, sess, b=0, screen=False):
    global_w = [node.weights() for node in W]
    if screen and b:
        new_w = screening(global_w, neighbor, b, method = screen) 
    else: #regular averaging update
        new_w = []
        for node in range(len(W)):
            ave = []
            for layer in range(len(global_w[node])):
                neighborhood = []
                for n in neighbor[node]:
                    neighborhood.append(global_w[n][layer])
                neighborhood = np.mean(neighborhood, axis = 0)
    #        print(neighborhood.shape)
                ave.append(neighborhood)
            new_w.append(ave)
    for node in range(len(new_w)):
        W[node].assign(new_w[node], sess)
    return W



if __name__ == "__main__":
    for bb in [4]:
        for NN in [1000]:
            r = 0
            para = experiment_parameters(agents=50, dataset='MNIST', localsize_N=NN, iteration=200,
                                          stepsize=1e-4, screen='Median', b=bb, Byzantine='random')
            local_set, test_data, test_label = data_distribution(para.dataset, para.M, para.N)

            W_0, graph = gen_graph(para.M, con_rate=50, b=para.b)
            neighbors = get_neighbor(graph) 
            #Initialization
            tf.reset_default_graph()

            w_nodes = []
            for node in range(para.M):
                w_nodes.append(CNN())

            sess = initialization(w_nodes)
            rec = []    
            beginning = time.time()  
            for iteration in range(para.T):
                #Byzantine nodes are the first b nodes
                w_nodes[:para.b] = Byzantine(w_nodes, para.b, local_set, sess, strategy=para.Byzantine)        
                #Communication 
                w_nodes = BRIDGE_communication(w_nodes, neighbors, sess, b=para.b, screen=para.screen)
                #local update
                w_nodes = local_update(w_nodes, sess, local_set, para.b)
                #test
                accuracy = [acc_test(w, test_data, test_label) for w in w_nodes[para.b:]]
                rec.append(accuracy)


                print(accuracy)
                print(iteration)    
                print(time.time()-beginning)
                beginning = time.time()
                with open('result/bN/strategy_Median_%d.pickle'%r, 'wb') as handle:
                    pickle.dump(rec, handle)

            sess.close()




Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

[0.1365, 0.1402, 0.1389, 0.1371, 0.1431, 0.1389, 0.1423, 0.1429, 0.139, 0.1421, 0.143, 0.1416, 0.1376, 0.1407, 0.1431, 0.1383, 0.1392, 0.1427, 0.1393, 0.1426, 0.1376, 0.1407, 0.1391, 0.1407, 0.1382, 0.1453, 0.1428, 0.1422, 0.1437, 0.1435, 0.1466, 0.1401, 0.1434, 0.1372, 0.1359, 0.138, 0.1417, 0.142, 0.1385, 0.141, 0.1445, 0.1406, 0.1412, 0.1428, 0.1406, 0.1423]
0
522.1562662124634
[0.1921, 0.1934, 0.192, 0.1879, 0.1978, 0.1936, 0.1948, 0.1944, 0.1935, 0.1987, 0.1958, 0.192, 0.1916, 0.1981, 0.1987, 0.1897, 0.1965, 0.1974, 0.193, 0.1952, 0.1944, 0.1916, 0.1891, 0.1925, 0.1903, 0.1964, 0.1929, 0.2018, 0.1938, 0.1972, 

[0.746, 0.7472, 0.751, 0.7512, 0.7481, 0.7487, 0.7496, 0.7468, 0.7515, 0.7506, 0.7447, 0.7533, 0.744, 0.7418, 0.7493, 0.7532, 0.7481, 0.7437, 0.7433, 0.7474, 0.7487, 0.7512, 0.7496, 0.7505, 0.7539, 0.7442, 0.7452, 0.7406, 0.7422, 0.754, 0.7394, 0.7494, 0.7474, 0.7497, 0.7442, 0.7528, 0.7467, 0.747, 0.7433, 0.7488, 0.7469, 0.7538, 0.7489, 0.7439, 0.7501, 0.7552]
19
409.223619222641
[0.7556, 0.7575, 0.7605, 0.7603, 0.7577, 0.7584, 0.7599, 0.7575, 0.7619, 0.7619, 0.7549, 0.763, 0.7556, 0.7503, 0.7605, 0.7626, 0.7596, 0.7533, 0.7527, 0.7572, 0.7581, 0.7586, 0.7607, 0.7619, 0.7627, 0.7533, 0.7546, 0.7492, 0.7529, 0.7633, 0.7504, 0.7571, 0.7583, 0.7604, 0.7515, 0.7622, 0.7567, 0.7573, 0.7538, 0.7626, 0.7599, 0.7634, 0.7612, 0.7543, 0.7609, 0.7659]
20
415.19906282424927
[0.7637, 0.767, 0.7678, 0.771, 0.7698, 0.7679, 0.7668, 0.7665, 0.7696, 0.7718, 0.7648, 0.7714, 0.7669, 0.7632, 0.7685, 0.7713, 0.7699, 0.7645, 0.7645, 0.7676, 0.7698, 0.7688, 0.7706, 0.7711, 0.7708, 0.7653, 0.7622, 0.7615, 0.7

[0.8747, 0.8756, 0.8732, 0.8743, 0.873, 0.8741, 0.8738, 0.8748, 0.8749, 0.8735, 0.8746, 0.8738, 0.8759, 0.8736, 0.8727, 0.8747, 0.8727, 0.8731, 0.8761, 0.8727, 0.8726, 0.8726, 0.8719, 0.8713, 0.8737, 0.8744, 0.8746, 0.8744, 0.8749, 0.8744, 0.8728, 0.8725, 0.8735, 0.8744, 0.875, 0.8737, 0.8736, 0.8743, 0.8739, 0.8738, 0.8728, 0.8717, 0.8734, 0.8736, 0.8728, 0.8739]
41
404.13690304756165
[0.8777, 0.878, 0.8763, 0.8778, 0.8768, 0.8773, 0.8764, 0.8771, 0.877, 0.8762, 0.8781, 0.8765, 0.8783, 0.8773, 0.8748, 0.8762, 0.8756, 0.8754, 0.8783, 0.8765, 0.8756, 0.8755, 0.8746, 0.8747, 0.8768, 0.8767, 0.8773, 0.8765, 0.8778, 0.8765, 0.8755, 0.8758, 0.8762, 0.878, 0.8768, 0.8755, 0.8759, 0.8762, 0.8767, 0.8771, 0.8764, 0.8765, 0.8756, 0.876, 0.8743, 0.8754]
42
411.0730149745941
[0.8808, 0.8806, 0.8794, 0.8789, 0.8793, 0.8806, 0.8785, 0.8788, 0.8796, 0.8778, 0.8807, 0.8791, 0.881, 0.8796, 0.878, 0.8788, 0.8792, 0.8781, 0.8812, 0.8786, 0.8775, 0.8782, 0.8796, 0.8778, 0.8793, 0.88, 0.8797, 0.8794, 0.87

[0.9131, 0.9135, 0.9113, 0.9127, 0.9119, 0.9137, 0.9133, 0.9131, 0.9131, 0.9128, 0.9126, 0.9125, 0.9126, 0.913, 0.9127, 0.9131, 0.9117, 0.9122, 0.9126, 0.9127, 0.9127, 0.9123, 0.9111, 0.9107, 0.9135, 0.912, 0.912, 0.9125, 0.9131, 0.9126, 0.9129, 0.9116, 0.9127, 0.9124, 0.9133, 0.9129, 0.9129, 0.9127, 0.913, 0.9127, 0.9115, 0.9126, 0.9131, 0.9123, 0.9119, 0.9107]
63
409.7896845340729
[0.9134, 0.9144, 0.9121, 0.9135, 0.913, 0.9149, 0.914, 0.9138, 0.9136, 0.9137, 0.9141, 0.9134, 0.9136, 0.9138, 0.9149, 0.9145, 0.9132, 0.9133, 0.914, 0.9139, 0.9135, 0.9137, 0.9121, 0.912, 0.9142, 0.9127, 0.9131, 0.9134, 0.9137, 0.9141, 0.9144, 0.9129, 0.9137, 0.914, 0.914, 0.9143, 0.9142, 0.9141, 0.9136, 0.9135, 0.9128, 0.9133, 0.9141, 0.913, 0.9133, 0.9117]
64
434.3585727214813
[0.9144, 0.9157, 0.9131, 0.9153, 0.9136, 0.9155, 0.9154, 0.9147, 0.9147, 0.9149, 0.915, 0.9136, 0.914, 0.9146, 0.9155, 0.9154, 0.9142, 0.9146, 0.9147, 0.9143, 0.9151, 0.9145, 0.9127, 0.9131, 0.9144, 0.9139, 0.9138, 0.9147, 0.9146, 

[0.9299, 0.93, 0.9282, 0.93, 0.9293, 0.9299, 0.9291, 0.9291, 0.9308, 0.9295, 0.9296, 0.9283, 0.9296, 0.9287, 0.9289, 0.9296, 0.9282, 0.9287, 0.9281, 0.9284, 0.9292, 0.929, 0.9288, 0.9284, 0.9305, 0.9276, 0.9293, 0.9288, 0.9285, 0.9307, 0.9294, 0.9291, 0.9291, 0.9296, 0.9291, 0.9289, 0.9298, 0.929, 0.929, 0.9291, 0.9273, 0.9302, 0.9293, 0.9283, 0.9282, 0.9281]
85
437.8606538772583
[0.9299, 0.9306, 0.9289, 0.9303, 0.9298, 0.9305, 0.9298, 0.9301, 0.931, 0.9304, 0.9303, 0.9288, 0.9303, 0.9294, 0.9295, 0.9296, 0.9295, 0.9291, 0.9293, 0.9291, 0.9296, 0.9293, 0.9296, 0.9284, 0.9306, 0.9282, 0.9295, 0.9297, 0.929, 0.9309, 0.93, 0.9294, 0.9294, 0.9301, 0.9295, 0.9292, 0.9302, 0.9301, 0.9293, 0.9296, 0.9279, 0.9309, 0.9293, 0.9292, 0.9288, 0.9293]
86
424.2361686229706
[0.931, 0.931, 0.9295, 0.9309, 0.93, 0.9307, 0.9299, 0.9307, 0.9312, 0.9312, 0.9304, 0.9295, 0.9307, 0.9299, 0.9297, 0.9303, 0.9297, 0.93, 0.9293, 0.9297, 0.9303, 0.9295, 0.9297, 0.9291, 0.931, 0.9285, 0.9301, 0.93, 0.93, 0.9311, 0

[0.9384, 0.938, 0.9373, 0.9379, 0.938, 0.9381, 0.9388, 0.9383, 0.9382, 0.9392, 0.9379, 0.9373, 0.9385, 0.9387, 0.9374, 0.939, 0.9376, 0.9387, 0.9379, 0.9386, 0.9385, 0.9381, 0.9378, 0.938, 0.9393, 0.9374, 0.9374, 0.9387, 0.9375, 0.939, 0.9385, 0.9378, 0.938, 0.9387, 0.9393, 0.9382, 0.9386, 0.9382, 0.9376, 0.9386, 0.9385, 0.9385, 0.9384, 0.9376, 0.938, 0.938]
107
422.1988377571106
[0.9383, 0.9382, 0.9385, 0.9381, 0.9383, 0.9386, 0.9392, 0.9384, 0.9383, 0.9393, 0.9383, 0.9378, 0.9384, 0.9391, 0.9378, 0.9395, 0.9383, 0.9394, 0.9384, 0.9389, 0.9388, 0.9385, 0.9381, 0.9382, 0.9395, 0.9379, 0.9378, 0.9387, 0.938, 0.9392, 0.9388, 0.938, 0.9384, 0.939, 0.9397, 0.9388, 0.9386, 0.9385, 0.9382, 0.9384, 0.9386, 0.9388, 0.9388, 0.938, 0.9386, 0.9383]
108
412.99586153030396
[0.9388, 0.9388, 0.9392, 0.9385, 0.9386, 0.9388, 0.9397, 0.939, 0.9387, 0.9399, 0.939, 0.9381, 0.9386, 0.9396, 0.9379, 0.9397, 0.9387, 0.9402, 0.939, 0.9393, 0.9391, 0.9388, 0.9379, 0.939, 0.9397, 0.9382, 0.9387, 0.9394, 0.9385, 

[0.9458, 0.9451, 0.9453, 0.9446, 0.9458, 0.9456, 0.9451, 0.9453, 0.9453, 0.9448, 0.9454, 0.9457, 0.9457, 0.9452, 0.9444, 0.945, 0.9454, 0.9446, 0.945, 0.9448, 0.9455, 0.945, 0.9456, 0.9453, 0.945, 0.9447, 0.9448, 0.9437, 0.9452, 0.9457, 0.945, 0.9452, 0.9451, 0.9459, 0.9458, 0.9445, 0.9448, 0.9445, 0.9455, 0.9452, 0.9454, 0.9452, 0.9453, 0.9451, 0.944, 0.9453]
129
420.3385236263275
[0.9461, 0.9455, 0.9454, 0.945, 0.9461, 0.9458, 0.9455, 0.9454, 0.9454, 0.9449, 0.9455, 0.9463, 0.9457, 0.9455, 0.945, 0.9449, 0.9459, 0.9447, 0.9453, 0.9449, 0.9458, 0.9452, 0.9454, 0.9457, 0.9454, 0.945, 0.945, 0.9443, 0.9456, 0.9459, 0.945, 0.9453, 0.945, 0.9459, 0.9463, 0.9451, 0.945, 0.945, 0.9456, 0.9456, 0.9457, 0.9452, 0.9454, 0.9453, 0.9449, 0.9452]
130
413.25918555259705
[0.9463, 0.9462, 0.9457, 0.945, 0.946, 0.9458, 0.946, 0.9456, 0.9453, 0.9458, 0.9455, 0.9463, 0.946, 0.9458, 0.9453, 0.9457, 0.946, 0.945, 0.9453, 0.9453, 0.9463, 0.9458, 0.9455, 0.9461, 0.9458, 0.9452, 0.945, 0.9444, 0.9456, 0.945

[0.9498, 0.9496, 0.9495, 0.9487, 0.9489, 0.9494, 0.9493, 0.9491, 0.9492, 0.9499, 0.9497, 0.9487, 0.9498, 0.9494, 0.9493, 0.9501, 0.9496, 0.9494, 0.9489, 0.949, 0.949, 0.9493, 0.949, 0.9495, 0.9492, 0.9491, 0.9489, 0.9491, 0.9498, 0.9498, 0.9493, 0.9487, 0.9492, 0.9495, 0.9494, 0.9494, 0.9492, 0.9493, 0.949, 0.9497, 0.9494, 0.9497, 0.9499, 0.9495, 0.9498, 0.9494]
151
413.83506512641907
[0.95, 0.9499, 0.9497, 0.9493, 0.9491, 0.9499, 0.9495, 0.9492, 0.9494, 0.9497, 0.9497, 0.9486, 0.9497, 0.9496, 0.9494, 0.9503, 0.9499, 0.9495, 0.9492, 0.9493, 0.9492, 0.9494, 0.9493, 0.9496, 0.9492, 0.9489, 0.9492, 0.9493, 0.9499, 0.9502, 0.9492, 0.9491, 0.9495, 0.9496, 0.9495, 0.9497, 0.9491, 0.9495, 0.9491, 0.9499, 0.9498, 0.9496, 0.9501, 0.9499, 0.95, 0.9498]
152
411.73421335220337
[0.95, 0.95, 0.9495, 0.9493, 0.9492, 0.9498, 0.9497, 0.9496, 0.9496, 0.9501, 0.9496, 0.949, 0.9499, 0.9497, 0.9493, 0.9506, 0.9501, 0.9497, 0.9491, 0.9497, 0.9495, 0.9498, 0.9493, 0.9501, 0.9494, 0.9491, 0.9496, 0.9497, 0.95

[0.9536, 0.9529, 0.9534, 0.9528, 0.9529, 0.9532, 0.9535, 0.9533, 0.954, 0.9528, 0.9533, 0.9529, 0.9534, 0.9535, 0.9533, 0.9543, 0.9532, 0.9538, 0.9529, 0.9529, 0.9533, 0.9533, 0.9531, 0.9537, 0.9527, 0.9534, 0.9529, 0.9533, 0.9528, 0.9541, 0.9537, 0.9528, 0.9529, 0.954, 0.9531, 0.9525, 0.9531, 0.9533, 0.9535, 0.9536, 0.9534, 0.9529, 0.9539, 0.9532, 0.9531, 0.9537]
173
414.6791377067566
[0.9538, 0.9532, 0.9539, 0.9527, 0.9527, 0.953, 0.9535, 0.9534, 0.9539, 0.9533, 0.9535, 0.9535, 0.9535, 0.9537, 0.9535, 0.9544, 0.9533, 0.9536, 0.9533, 0.9529, 0.9533, 0.9533, 0.9536, 0.9539, 0.9529, 0.9534, 0.9529, 0.9535, 0.9534, 0.9542, 0.954, 0.953, 0.9531, 0.954, 0.9533, 0.9528, 0.9534, 0.9534, 0.9533, 0.954, 0.9535, 0.9532, 0.954, 0.9534, 0.9529, 0.9542]
174
413.78194093704224
[0.954, 0.9538, 0.9539, 0.9531, 0.953, 0.9534, 0.9539, 0.9534, 0.9543, 0.9534, 0.9535, 0.9532, 0.9537, 0.9538, 0.9538, 0.9545, 0.9534, 0.9537, 0.9532, 0.9532, 0.9533, 0.9537, 0.9536, 0.9538, 0.9536, 0.9538, 0.9531, 0.9541, 0.

[0.9565, 0.9562, 0.9561, 0.9554, 0.9551, 0.9561, 0.9564, 0.9562, 0.9562, 0.9566, 0.9562, 0.9563, 0.9567, 0.9561, 0.9569, 0.957, 0.9566, 0.9568, 0.9554, 0.9562, 0.9565, 0.9559, 0.9566, 0.9566, 0.9562, 0.9559, 0.9562, 0.9564, 0.9556, 0.9564, 0.9568, 0.9565, 0.9565, 0.9572, 0.9571, 0.9561, 0.9558, 0.956, 0.9554, 0.9561, 0.956, 0.9564, 0.9571, 0.9563, 0.9568, 0.9561]
195
399.5886890888214
[0.9566, 0.9565, 0.9569, 0.9558, 0.9555, 0.9562, 0.9565, 0.956, 0.9562, 0.9568, 0.9563, 0.9563, 0.9567, 0.9564, 0.9569, 0.9569, 0.9569, 0.9567, 0.9553, 0.9563, 0.9567, 0.9562, 0.9567, 0.9571, 0.9564, 0.9559, 0.9562, 0.9565, 0.9558, 0.9569, 0.9569, 0.9567, 0.9563, 0.9572, 0.9572, 0.9563, 0.956, 0.9564, 0.9555, 0.9566, 0.9559, 0.9568, 0.9571, 0.9563, 0.9572, 0.956]
196
398.633985042572
[0.957, 0.9561, 0.957, 0.9564, 0.9558, 0.9567, 0.9567, 0.9562, 0.9564, 0.9569, 0.9563, 0.9564, 0.9567, 0.9564, 0.9572, 0.957, 0.9567, 0.9568, 0.9557, 0.9564, 0.9572, 0.9566, 0.9567, 0.9571, 0.9565, 0.9561, 0.9566, 0.9565, 0.9

In [2]:
import numpy as np
from pre_proc import data_prep
from CNN_model import CNN
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
    def __init__(self, agents, dataset, localsize_N, iteration, batchsize=0, stepsize=1, 
                 screen=False, b=0, Byzantine='random'):
        self.dataset = dataset
        self.M = agents
        self.T = iteration
        self.screen = screen
        self.b = b
        self.stepsize = stepsize
        self.batchsize = batchsize
        self.N = localsize_N
        self.Byzantine = Byzantine
        if self.batchsize >  self.N:
            raise ValueError("Batch size is large than local datasize")


def data_distribution(dataset, nodes_M, sample_N):#samples will be distributed evenly to each node
    distributed_data, test_data, test_label = data_prep(dataset, nodes_M, nodes_M * sample_N, one_hot=True)
    return distributed_data, test_data, test_label

def initialization(W):
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    weight = W[0].weights()
    for w in W:
        w.assign(weight, sess)
    return sess


def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
    
    re = 1                      # regenerate if graph assumption not satisfied
    while re:
        graph = []
        for _ in range(nodes):
            graph.append([])
        for row in range(nodes):
            graph[row].append(1)
            for col in range(row + 1, nodes):
                d = random.randint(1, 100)
                if d < con_rate:
                    graph[row].append(1)     #form symmetric matrix row by row
                    graph[col].append(1)
                else:
                    graph[row].append(0)
                    graph[col].append(0)
        d_max = 0
        for row in graph:
            if sum(row) > d_max:
                d_max = sum(row)
        w = [row[:] for row in graph]
        for ind, row in enumerate(w):
            d = sum(row)
            w[ind] = [col/d_max for col in row]
            w[ind][ind] = 1 - (d - 1) / d_max
        if all([sum(row) >= 2 * b + 1 for row in graph]):
            re = 0
    return w, graph

def get_neighbor(G):
    neighbor_list = []
    for node in G:
        neighbor = []
        for ind, n in enumerate(node):
            if n == 1:
                neighbor.append(ind)
        neighbor_list.append(neighbor)
    return neighbor_list

def Byz_nonfaulty(W, b, data, sess, scaling=1):
    for model, sample, label in zip(W[:b], data.dist_data[:b], data.dist_label[:b]):        
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.keep_prob : 0.5})
    return W[:b]        


def Byz_random(W, b, data, sess, scaling=1):
    for model in W[:b]:
        weight = model.weights()
        new = [np.random.rand(*layer.shape) * scaling for layer in weight]
        model.assign(new, sess)
    return W[:b]

def Byz_flip(W, b, data, sess, scaling=1):
    Byz_data = data.dist_data[:b] * (-1) * scaling
    for model, sample, label in zip(W[:b], Byz_data, data.dist_label[:b]):        
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.keep_prob : 0.5})
    return W[:b]  

def Byz_omniscient(W, b, data, sess, scaling=1):
    all_weights = [node.weights() for node in W[b:]]
    ave = []
    for layer in len(all_weights[b]):
        one_layer = [node[layer] for node in all_weights]
        one_layer = np.mean(one_layer, axis=0)           
        ave.append(one_layer)
    for node in W[:b]:
        node.assign(ave, sess)
    return W[:b]

def Byzantine(W, b, data, sess, strategy='nonfaulty', scaling=1):
    strategies = {'nonfaulty': Byz_nonfaulty, 'random': Byz_random, 'flip': Byz_flip, 'omniscient': Byz_omniscient}    
    w_B = strategies[strategy](W, b, data, sess)
    return w_B

    
def acc_test(model, t_data, t_label):
    acc = model.accuracy.eval(feed_dict={
            model.x:t_data, model.y_: t_label, model.keep_prob: 1.0})
    return acc
    
def local_update(W, sess, data, b):
    for model, sample, label in zip(W[b:], data.dist_data[b:], data.dist_label[b:]):        
        sess.run(model.train_step, feed_dict={model.x: sample, model.y_: label, model.keep_prob : 0.5})
    return W

def Trimmed_mean(W, neighbor, b):
    new_w = []
    for node in range(len(W)):
        ave = []
        for layer in range(len(W[node])):
            neighborhood = []
            for n in neighbor[node]:
                neighborhood.append(W[n][layer])
            neighborhood = np.sort(neighborhood, axis = 0)
            neighborhood = neighborhood[b : -b]
            neighborhood = np.mean(neighborhood, axis = 0) 
#        print(neighborhood.shape)
            ave.append(neighborhood)
        new_w.append(ave)   
    return new_w

def Krum(W, neighbor, b):
    new_w = []   
    #score is calculated for only the last layer
    for node in range(len(W)):
        score = []
        neighborhood = []
        neighbor_list = []
        for n in neighbor[node]:
            neighborhood.append(W[n][-1])
            neighbor_list.append(n)
        for g in neighborhood:
            dist = [np.abs(other - g) for other in neighborhood]
            dist = np.sort(dist)
            score.append(np.sum(dist[:(len(neighborhood) - b - 2)]))
        ind = score.index(min(score))
        new_w.append(W[neighbor_list[ind]]) 
    return new_w    



def Median(W, neighbor, b):
    new_w = []
    for node in range(len(W)):
        ave = []
        for layer in range(len(W[node])):
            neighborhood = []
            for n in neighbor[node]:
                neighborhood.append(W[n][layer])
            neighborhood = np.median(neighborhood, axis = 0)
#        print(neighborhood.shape)
            ave.append(neighborhood)
        new_w.append(ave)   
    return new_w


def Bulyan(W, neighbor, b):
    new_w = []   
    #score is calculated for only the last layer
    for node in range(len(W)):
        neighborhood = []
        neighbor_list = []
        for n in neighbor[node]:
            neighborhood.append(W[n][-1])
            neighbor_list.append(n)
        M = len(neighbor_list)
        second_round = []
        for _ in range(M - 2 * b):
            score = []
            for g in neighborhood:
                dist = [np.abs(other - g) for other in neighborhood]
                dist = np.sort(dist)
                score.append(np.sum(dist[:(len(neighborhood) - b - 2)]))
            ind = score.index(min(score))
            second_round.append(W[neighbor_list[ind]])
            del neighborhood[ind]
            del neighbor_list[ind]
        ave = []
        for layer in range(len(second_round[0])):
            current_layer = [node[layer] for node in second_round]
            current_layer = np.sort(current_layer, axis = 0)
            current_layer = current_layer[b : -b]
            current_layer = np.mean(current_layer, axis = 0)
            ave.append(current_layer)
        new_w.append(ave)          
    return new_w

def screening(W, neighbor, b, method='Trim'):
    method_list = {'Trim': Trimmed_mean, 'Median': Median, 'Krum': Krum, 'Bulyan': Bulyan}
    aggregated = method_list[method](W, neighbor, b)
    return aggregated


def BRIDGE_communication(W, neighbor, sess, b=0, screen=False):
    global_w = [node.weights() for node in W]
    if screen and b:
        new_w = screening(global_w, neighbor, b, method = screen) 
    else: #regular averaging update
        new_w = []
        for node in range(len(W)):
            ave = []
            for layer in range(len(global_w[node])):
                neighborhood = []
                for n in neighbor[node]:
                    neighborhood.append(global_w[n][layer])
                neighborhood = np.mean(neighborhood, axis = 0)
    #        print(neighborhood.shape)
                ave.append(neighborhood)
            new_w.append(ave)
    for node in range(len(new_w)):
        W[node].assign(new_w[node], sess)
    return W



if __name__ == "__main__":
    for bb in [4]:
        for NN in [1000]:
            r = 1
            para = experiment_parameters(agents=1, dataset='MNIST', localsize_N=1000, iteration=200,
                                          stepsize=1e-4, screen='Median', b=0, Byzantine='random')
            local_set, test_data, test_label = data_distribution(para.dataset, para.M, para.N)

#             W_0, graph = gen_graph(para.M, con_rate=50, b=para.b)
#             neighbors = get_neighbor(graph) 
            #Initialization
            tf.reset_default_graph()

            w_nodes = []
            for node in range(para.M):
                w_nodes.append(CNN())

            sess = initialization(w_nodes)
            rec = []    
            beginning = time.time()  
            for iteration in range(para.T):
                #Byzantine nodes are the first b nodes
#                 w_nodes[:para.b] = Byzantine(w_nodes, para.b, local_set, sess, strategy=para.Byzantine)        
                #Communication 
#                 w_nodes = BRIDGE_communication(w_nodes, neighbors, sess, b=para.b, screen=para.screen)
                #local update
                w_nodes = local_update(w_nodes, sess, local_set, para.b)
                #test
                accuracy = [acc_test(w, test_data, test_label) for w in w_nodes[para.b:]]
                rec.append(accuracy)


                print(accuracy)
                print(iteration)    
                print(time.time()-beginning)
                beginning = time.time()
                with open('result/bN/strategy_local.pickle', 'wb') as handle:
                    pickle.dump(rec, handle)

            sess.close()




[0.0924]
0
7.272955417633057
[0.1183]
1
7.222317457199097
[0.1488]
2
7.15214991569519
[0.196]
3
7.046859502792358
[0.2445]
4
7.1971518993377686
[0.2898]
5
7.225222826004028
[0.3206]
6
7.173662424087524
[0.3572]
7
7.099680662155151
[0.403]
8
7.237814426422119
[0.4515]
9
7.1956377029418945
[0.509]
10
7.2181971073150635
[0.5626]
11
7.269834518432617
[0.6159]
12
7.321471929550171
[0.6569]
13
7.242762327194214
[0.6829]
14
7.130965709686279
[0.6983]
15
7.4317731857299805
[0.7115]
16
7.641840219497681
[0.7206]
17
7.787333250045776
[0.7292]
18
7.7015368938446045
[0.7394]
19
7.31395149230957
[0.7502]
20
7.275349378585815
[0.7619]
21
7.2502830028533936
[0.7728]
22
7.1996870040893555
[0.7828]
23
7.2432777881622314
[0.791]
24
7.178111791610718
[0.7971]
25
7.322976350784302
[0.8006]
26
7.278871297836304
[0.8053]
27
7.208210468292236
[0.809]
28
7.138987064361572
[0.8119]
29
7.173076868057251
[0.8153]
30
7.0888683795928955
[0.8194]
31
7.130006790161133
[0.824]
32
7.081336975097656
[0.8278]
33
7.09441

In [ ]:
#checkpoint
class byrdie_checkpt(weights, save, iteration):
    def __ini__(self):
        self.weights = weights
        self.save = save
        self.iteration = iteration
        
class byrdie_initialization(exp_parameters, graph, neighbors, data):
    def __ini__(self):
        self.parameters = exp_parameters
        self.graph = graph
        self.neighbors = neighbors
        self.data = data



In [10]:
# Decentralized learning with ByRDiE

# Specify M nodes, N local samples, dataset, failure type, screening type, stepsize, connection rate, T iterations,
# b Byzantine agents

import numpy as np
from pre_proc import data_prep
from linear_classifier import linear_classifier
import tensorflow as tf
import time
import pickle
import random


class experiment_parameters:
   def __init__(self, agents, dataset, localsize_N, iteration, stepsize=1e-4, 
                screen=False, b=0, Byzantine='random', connection_rate=50):
       self.dataset = dataset
       self.M = agents
       self.T = iteration
       self.screen = screen
       self.b = b
       self.stepsize = stepsize
       self.N = localsize_N
       self.Byzantine = Byzantine
       self.con_rate = connection_rate

def gen_graph(nodes, con_rate, b=0):          # connecting rate between 1-100
   
   re = 1                      # regenerate if graph assumption not satisfied
   while re:
       graph = []
       for _ in range(nodes):
           graph.append([])
       for row in range(nodes):
           graph[row].append(1)
           for col in range(row + 1, nodes):
               d = random.randint(1, 100)
               if d < con_rate:
                   graph[row].append(1)     #form symmetric matrix row by row
                   graph[col].append(1)
               else:
                   graph[row].append(0)
                   graph[col].append(0)
       d_max = 0
       for row in graph:
           if sum(row) > d_max:
               d_max = sum(row)
       w = [row[:] for row in graph]
       for ind, row in enumerate(w):
           d = sum(row)
           w[ind] = [col/d_max for col in row]
           w[ind][ind] = 1 - (d - 1) / d_max
       if all([sum(row) >= 2 * b + 1 for row in graph]):
           re = 0
   return w, graph   

def get_neighbor(G):
   neighbor_list = []
   for node in G:
       neighbor = []
       for ind, n in enumerate(node):
           if n == 1:
               neighbor.append(ind)
       neighbor_list.append(neighbor)
   return neighbor_list

def one_hot(label):
   l_oh = []
   for i in label:
       new_l = [0] * 10
       new_l[i] = 1
       l_oh.append(new_l)
   return l_oh


def initialization():
   sess = tf.InteractiveSession()
   sess.run(tf.global_variables_initializer())
   return sess


def screen(grad, b, method):
   screened = Byzantine_algs[method](grad, b)
   return screened
   
def acc_test(model, t_data, t_label):
   acc = model.accuracy.eval(feed_dict={
           model.x:t_data, model.y_: t_label,})
   return acc
   
def communication_w(W, neighbor, p, sess, b=0, screen=False):
   _w = [node.weights()[0].reshape([7840,1]) for node in W]
   _b = [node.weights()[1] for node in W]
   ave = []
   for neighbor_list in neighbor:
       neighborhood = [_w[n][p] for n in neighbor_list]

       if screen:
           neighborhood = np.sort(neighborhood, axis = 0)
           neighborhood = neighborhood[b : -b]
       neighborhood = np.mean(neighborhood, axis = 0)
   #        print(neighborhood.shape)
       ave.append(neighborhood)
   for scalar, node in zip(ave, _w):
       node[p] = scalar
   
   for node, ww, bb in zip(W, _w, _b):
       node.assign([ww.reshape([784, 10]), bb], sess) 
       
def communication_b(W, neighbor, p, sess, b=0, screen=False):
   _w = [node.weights()[0] for node in W]
   _b = [node.weights()[1] for node in W]
   ave = []
   for neighbor_list in neighbor:
       neighborhood = [_b[n][p] for n in neighbor_list]

       if screen:
           neighborhood = np.sort(neighborhood, axis = 0)
           neighborhood = neighborhood[b : -b]
       neighborhood = np.mean(neighborhood, axis = 0)
   #        print(neighborhood.shape)
       ave.append(neighborhood)
   for scalar, node in zip(ave, _b):
       node[p] = scalar
   
   for node, ww, bb in zip(W, _w, _b):
       node.assign([ww, bb], sess) 

def node_update_w(W, data, p, sess, stepsize=1e-4):        
   for model, sample, label in zip(W, data.dist_data, data.dist_label):
       g = sess.run(model.gradient_w, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
       new_g = np.zeros([7840, 1])
       new_g[p] = g.reshape([7840, 1])[p]
       sess.run(model.local_update_w, feed_dict={model.gradient_port_w: new_g.reshape([784,10]), model.stepsize: stepsize})
       
def node_update_b(W, data, p, sess, stepsize=1e-4):        
   for model, sample, label in zip(W, data.dist_data, data.dist_label):
       g = sess.run(model.gradient_b, feed_dict={model.x: sample, model.y_: label, model.stepsize: stepsize})
       new_g = np.zeros([10])
       new_g[p] = g[p]
       sess.run(model.local_update_b, feed_dict={model.gradient_port_b: new_g, model.stepsize: stepsize})

#checkpoint
class byrdie_checkpt:
    def __init__(self, weights, save, iteration):
        self.weights = weights
        self.save = save
        self.iteration = iteration
        
    def pickle(self, r):
        with open('./checkpt/ByRDiE_checkpt_current_%d.pickle'%r, 'wb') as handle:
            pickle.dump(self, handle)
        
class byrdie_initialization:
    def __init__(self, exp_parameters, graph, neighbors, local_data, test_data, test_label):
        self.parameters = exp_parameters
        self.graph = graph
        self.neighbors = neighbors
        self.local_data = local_data
        self.test_data = test_data
        self.test_label = test_label
    def save(self, r):
        with open('./checkpt/ByRDiE_checkpt_ini_%d.pickle'%r, 'wb') as handle:
            pickle.dump(self, handle)
        
def chekpt_read(r):
    with open('./checkpt/ByRDiE_checkpt_ini_%d.pickle'%r, 'rb') as handle: 
        ini = pickle.load(handle)
    with open('./checkpt/ByRDiE_checkpt_current_%d.pickle'%r, 'rb') as handle: 
        current = pickle.load(handle)
        
    return ini.parameters, ini.graph, ini.neighbors, ini.local_data, ini.test_data, ini.test_label, current.weights, current.save, current.iteration

if __name__ == "__main__":
    r = 1
    try:
        para, graph, neighbors, local_set, test_data, test_label, weights, save, t = chekpt_read(r)
        loaded = True
    except:        
        para = experiment_parameters(agents=10, dataset='MNIST', localsize_N=40, iteration=100,
                                  screen=False, b=0, Byzantine='random', stepsize = 1e-1)
        loaded = False
    #Generate the graph
        W_0, graph = gen_graph(para.M, para.con_rate, para.b)    
        local_set, test_data, test_label = data_prep(para.dataset, para.M, para.N, one_hot=True)
        neighbors = get_neighbor(graph)
        checkpt_ini = byrdie_initialization(para, graph, neighbors, local_set, test_data, test_label)
        save = []
        t = 0
        checkpt_ini.save(r)
    #Initialization
    tf.reset_default_graph()
    w_nodes = [linear_classifier(stepsize = para.stepsize) for node in range(para.M)]    
    sess = initialization()
    if loaded:
        for w, node in zip(weights, w_nodes):
            node.assign(w, sess)
    
    for iteration in range(t + 1, para.T):
        
        step = para.stepsize/(iteration+1)
        for p in range(784):
       #Communication 
           communication_w(w_nodes, neighbors, p, sess, para.b, screen = para.screen)
       #node update using Adam    
           node_update_w(w_nodes, local_set, p, sess, stepsize=step)
           if p%200 == 199:  
                
           #test over all test data
                accuracy = [acc_test(node, test_data, test_label) for node in w_nodes]
                print(accuracy)
                save.append(accuracy)
        for p in range(10):
            communication_b(w_nodes, neighbors, p, sess, para.b, screen = para.screen)
            node_update_b(w_nodes, local_set, p, sess, stepsize=step)
        accuracy = [acc_test(node, test_data, test_label) for node in w_nodes]
        print(accuracy)
        print(iteration)
        save.append(accuracy)
        with open('./result/ByRDiE_faultless_%d.pickle'%r, 'wb') as handle:
            pickle.dump(save, handle) 
        weights = [node.weights() for node in w_nodes]
        checkpt_current = byrdie_checkpt(weights, save, iteration)
        checkpt_current.pickle(r)
    sess.close()


[0.105, 0.1496, 0.088, 0.1129, 0.1062, 0.0738, 0.1131, 0.1706, 0.068, 0.1129]
[0.105, 0.1496, 0.088, 0.1129, 0.1062, 0.0738, 0.1131, 0.1706, 0.068, 0.1129]


KeyboardInterrupt: 